## NG Solve start

defining the geometry

In [33]:
from ngsolve import *
from ngsolve.webgui import *
from netgen.geom2d import *
from netgen.occ import *

a = 28
b = 4
A = (0,0)


shape = MoveTo(0,0).Rectangle(a,b).Face()
pillar = Circle((10,2),0.5).Face()

# p1 = (0,0)
# p2 = (0,b)
# p3 = (a,0)
# p4 = (a,b)

# curves = [[["line",p1,p2],"source"],
#           [["line",p3,p4],"sink"]]


# geo = SplineGeometry()

# pnts =[(0,0),
#        #(0,0,0.05), # define a local mesh refinement for one point
#        (1,0),
#        (1,0.5),
#        (1,1),
#        (0.5,1),
#        (0,1)]

# p1,p2,p3,p4,p5,p6 = [geo.AppendPoint(*pnt) for pnt in pnts]

# curves = [[["line",p1,p2],"bottom"],
#           [["line",p2,p3],"right"],
#           [["spline3",p3,p4,p5],"curve"],
#           [["line",p5,p6],"top"],
#           [["line",p6,p1],"left"]]

# [geo.Append(c,bc=bc) for c,bc in curves]



shape.edges.name = "shore"      # TODO declaring SOURCE and SINK is still not done
pillar.edges.name = "wall"


rect = shape - pillar 


geo = OCCGeometry(rect,dim=2)

# mesh = Mesh(unit_square.GenerateMesh(maxh=0.2))
# mesh.GetBoundaries()

creating a mesh

In [34]:
from ngsolve import *
from ngsolve.webgui import Draw
mesh = Mesh(geo.GenerateMesh(maxh=2))
mesh.Curve(3)
Draw(mesh)

#Draw(b,mesh,"Grundhöhe")

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

Vectorial approximation space

In [35]:
#order = 2
#fes = L2(mesh,order=order)**3

fes = H1(mesh, order=2, dirichlet="bottom|right")
fes.ndof  # number of unknowns in this space


312

initial & boundary conditions

In [36]:

u = fes.TrialFunction()
v = fes.TestFunction()
gfu = GridFunction(fes)

#short version:
#u,v = fes.TnT()         # U is solution-function, V is test-function


define the solution of poisson equation: f!!!

we have a pde-problem - here its a poisson equation with
- laplace(u) = f_pde              on Omega
- u = 0                           on boundary of Omega

In [37]:
f_pde = 5*x*sin(100*x*y)

assemble bilinear/linear forms

In [38]:

a = BilinearForm(fes)
a += grad(u)*grad(v)*dx    # here was the lemma of green used, grad(u)*grad(v) = laplace(u) * v 
a.Assemble()

f = LinearForm(fes)
f += f_pde*v*dx
f.Assemble()

#short version:
#a = BilinearForm(grad(u)*grad(v)*dx).Assemble()
#f = LinearForm(x*v*dx).Assemble()


solve the system

In [39]:
gfu.vec.data = \
    a.mat.Inverse(freedofs=fes.FreeDofs()) * f.vec
Draw(gfu);


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…